# core

> Fill in a module description here

In [ ]:
#| default_exp loss

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def FocalLoss(): pass

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class focalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super(focalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, y):
        p = torch.sigmoid(inputs)

        bce_loss = F.binary_cross_entropy_with_logits(inputs, y, reduction='none')

        p_t = p * y + (1 - p) * (1 - y)
        focal_weight = (1 - p_t) ** self.gamma

    
        alpha_t = self.alpha * y + (1 - self.alpha) * (1 - y)
        bce_loss = alpha_t * bce_loss

        loss = focal_weight * bce_loss
        return loss.mean()

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  # Can be a scalar or a tensor of shape (num_classes,)

    def forward(self, inputs, targets):
        # 1. Compute standard BCE loss
        # Use binary_cross_entropy_with_logits for numerical stability
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')

        # 2. Calculate p_t
        # p_t = exp(-bce_loss) is numerically more stable than calculating sigmoid manually
        p_t = torch.exp(-bce_loss)

        # 3. Calculate Focal Weight
        focal_weight = (1 - p_t) ** self.gamma

        # 4. Handle Alpha (Class Balancing)
        if self.alpha is not None:
            # If alpha is a tensor, it must match the number of classes in targets
            # targets shape: (batch_size, num_classes)
            if isinstance(self.alpha, (float, int)):
                alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            else:
                # Assuming self.alpha is a tensor of shape (num_classes,)
                alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            
            bce_loss = alpha_t * bce_loss

        # 5. Final Loss
        loss = focal_weight * bce_loss
        
        return loss.mean()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()